# **Analyzing CIA Factbook Data Using SQL**

### **Context:**  
In this project, we will work with the data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) which contains demographic information about population growth and area. We'll use SQL to analyze this data.

**Note:** This code was written on Jupyter Notebook.  
**Language:** SQL.

**Connect Jupyter Notebook to the database file**

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

**Explore the data**

The database contains tables. In order to return information about the table, we write the following query:

In [5]:
%%sql
SELECT *
  FROM sqlite_master
    WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


The dataframe above shows information about two tables named sqlite_sequence and facts. Next we'll write a query to return information about the facts table and display the first five rows.

In [6]:
%%sql
SELECT *
  FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The table contains information about countries and their respective populations and areas. I did some research about the descriptions of the columns that you can find below:
- id: id code.
- code: code name.
- name: the name of the country.
- area: the country's total area (both land and water).
- area_land: the country's land area in square kilometers.
- area_water: the country's water area in square kilometers.
- population: the country's population.
- population_growth: the country's population growth as a percentage.
- birth_rate: the country's birth rate, or the number of births per year per 1000 people.
- death_rate: the country's death rate, or the number of deaths per year per 1000 people.
- migration_rate: the country's migration rate as a percentage.

Next, we will write a single query that returns the following information:
- Minimum population.
- Maximum population.
- Minimum population growth.
- Maximum population growth.

In [7]:
%%sql
SELECT MIN(population) AS minimum_population, 
       MAX(population) AS maximum_population,
       MIN(population_growth) AS minimum_population_growth, 
       MAX(population_growth) AS maximum_population_growth
    FROM facts;

Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,7256490011,0.0,4.02


The summary statistics reveals two interesting things:
- There is a country with a population of 0.
- There is a country with a population of 7256490011 (or 7.2 billion people).

Next, we'll write a query that returns countries with the minimum population.

In [8]:
%%sql
SELECT name, population
 FROM facts
WHERE population == (SELECT MIN(population)
                      FROM facts);

Done.


name,population
Antarctica,0


Likewise, we'll write a query that returns countries with maximum population.

In [9]:
%%sql
SELECT name, population
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts); 

Done.


name,population
World,7256490011


It seems like there is a row for the whole world which explains the number of population of 7256490011. We also have a row for Antarctica which is why we have zero population.
Now that we know this we need to recalculate the summary statistics and exclude the row for the whole world.

In [10]:
%%sql
SELECT MIN(population) AS minimum_population, 
       MAX(population) AS maximum_population,
       MIN(population_growth) AS minimum_population_growth, 
       MAX(population_growth) AS maximum_population_growth
    FROM facts
 WHERE population <> (SELECT MAX(population)
                        FROM facts); 

Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,1367485388,0.0,4.02


Calculate the average value for the population and area columns:

In [11]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
    FROM facts
  WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


Finally, we'll write a query that identifies the following:
- Above average values for population.
- below average values for area.

In [12]:
%%sql
SELECT name, population, area
   FROM facts
 WHERE (population > (SELECT AVG(population)
                         FROM facts
                        WHERE name <> 'World')) 
   AND (area < (SELECT AVG(area)
                   FROM facts
                  WHERE name <> 'World'));

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370
